# Introducción

El aprendizaje por refuerzo explora una aproximación *computacional* al aprendizaje por interacción. De la misma manera que cuando aprendemos a conducir estamos atentos a cómo reacciona nuestro entorno a nuestras acciones y buscamos maneras de influenciarlo a través de nuestro comportamiento, el aprendizaje por refuerzo estudia cómo agentes computacionales pueden desarrollar comportamientos inteligentes a través de este tipo de interacción.

## Aprendizaje por refuerzo

El aprendizaje por refuerzo, de ahora en adelante **axr**, consiste en aprender qué hacer (cómo relacionar situaciones y acciones) con el objetivo de maximizar una recompensa numérica. En ningún momento especificamos qué acciones debe tomar un *agente*, sino que le dejamos descubrir cuales son las que le darán una mayor recompensa. En la mayoría de situaciones, una acción no solo afectará a la recompensa inmediata, si no que tendrá un efecto en todas las situaciones futuras. Estas dos propiedades, búsqueda por prueba y error y futuras recompensas, son las más importantes del axr.

Para formalizar el problema del axr utilizamos ideas del campo de la teoría de sistemas dinámicos, en concreto el control óptimo de procesos de Markov incompletos. La idea básica consiste en aprender los aspectos más importantes sobre el problema real al que nuestro agente se enfrenta a través de la interacción con el entrono para conseguir su objetivo. El agente tiene que ser capaz de *precibir* su entorno y de llevar a cabo acciones que afecten a su estado. También necesita uno o varios objetivos relacionados con el estados del entorno. Un proceso de decisión de Markov incluyte estos tres aspectos: percepción, acción y objetivo. Cualquier método que sea capaz de resolver este tipo de problemas se considera como un método de axr.

El axr está considerado como un paradigma del machine learning diferente al aprendizaje supervisado y no supervisado. Se diferencia del aprendizaje supervisado en que no siempre será posible obtener ejemplos del comportamiento deseado para nuestro agente en cualquier tipo de situación en la que se pueda encontrar, por lo que deberá ser capaz de aprender de su propia experiencia. Por otro lado, se diferencia del aprendizaje no supervisado ya que éste no es capaz por si mismo de resolver el problema de maximización de la recompensa.

El principal problema al que nos enfrentamos en el axr es el balance entre **exploración** y **explotación**. Para obtener la máxima recompensa, un agente podría escoger aquellas acciones que ya conoce que y que le direon buenos resultados. Sin embargo, el hecho de explorar nuevas acciones podría, eventualmente, dar mucho mejor resultado. Así pues, nuestro agente tiene que ser capaz de explotar su conocimiento para obtener recompensa pero también de explorar para descubrir mejores acciones. El problema es que ninguna de las dos aproximaciones puede llevarse a cabo de manera independiente para resolver un problema. Un agente debe probar muchas acciones y, poco a poco, favorecer aquellas que parezcan ser mejores. Este problema sigue sin estar resuelto.

Otra propiedad importante que diferencia al axr de otro métodos es su capacidad de considerar todo el dominio del problema de un agente interactuando con su entrono, y no pequeñas partes o sub-tareas que puedan resolverse de manera independiente para llevar al objetivo final.

## Ejemplos

Algunos ejemplos interesantes de aplicaciones del axr son:

- Agentes que son capaces de jugar a juegos: ajedrez, go, atari, starcraft ...
- Sistemas de control adaptativo en entornos industriales: refinerías, fábricas, cadenas de montaje, ...
- Robótica
- Conducción autónoma: el coche recibe información de su entorno a través de sus cámaras y sensores y ejecuta comandos para acelerar, frenar, girar el volante, ...

Como vemos, todos los ejemplos comparten la existencia de un agente en constante interacción con su entorno para lograr un objetivo (a pesar de la posible incertidumbre). Tomar una decisión puede afectar a las acciones y oportunidades futuras, por lo que la elección de una accion requiere capacidad de planificación y predicción. De todas las formas de inteligencia computacional, el axr es el que más se asemeja a la forma en la que personas y animales actuamos.